In [3]:
from pyspark.sql import SparkSession
    
import os

# setup arguments
os.environ['kafka1'] = '34.220.52.64'
os.environ['kafka2'] = '52.35.125.165'
os.environ['schema-registry'] = '52.35.125.165'


os.environ['PYSPARK_PYTHON'] = "/opt/workspace/env3/bin/python3.9"

spark = SparkSession.builder.appName('pyspark-notebook')\
    .master("spark://spark-master:7077")\
    .config("spark.jars", "scalaSpark-assembly-1.0.jar") \
    .config('spark.jars.packages', 'com.amazonaws:aws-java-sdk-bundle:1.11.974,org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-sql_2.12:3.0.0,org.apache.kafka:kafka-clients:2.8.0,org.apache.spark:spark-avro_2.12:3.0.0,io.delta:delta-core_2.12:0.8.0,org.apache.spark:spark-streaming-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-streaming_2.12:3.0.0')\
    .config("spark.executor.memory", "512m")\
    .config("spark.cores.max", "12")\
    .config("spark.driver.port", "4040")\
    .config("spark.archives","venv1.zip")\
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .getOrCreate()

Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.amazonaws#aws-java-sdk-bundle added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-sql_2.12 added as a dependency
org.apache.kafka#kafka-clients added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
io.delta#delta-core_2.12 added as a dependency
org.apache.spark#spark-streaming-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-streaming_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0e931605-49af-43af-a3c7-9e68fea628f3;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk-bundle;1.11.974 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-pr

In [4]:
import ipywidgets as widget
l = widget.Label('consumerGroup')
display(l)
consumerGroup = widget.Text()
display(consumerGroup)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [5]:
consumerGroup.value

''

In [6]:
kafka = spark \
  .read \
  .format("kafka") \
.option("kafka.bootstrap.servers", "34.220.52.64:9092,52.35.125.165:9092")\
.option("subscribePattern", "qatest12nonfes.public.field_data_+(1[0-2]|[1-9])_202[0-9]")\
.option("startingOffsets", "earliest")\
.option("maxOffsetsPerTrigger", 1)\
.option("failOnDataLoss", False)\
.load()

22/03/10 07:51:00 WARN KafkaSourceProvider: maxOffsetsPerTrigger option ignored in batch queries


In [7]:
kafka.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
pip install python-schema-registry-client confluent_kafka avro

Note: you may need to restart the kernel to use updated packages.


In [9]:
from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col
import pyspark.sql.functions as fn
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from schema_registry.client import SchemaRegistryClient, schema
import json
import subprocess
import sys
import json
import sys
from pyspark import SparkConf
from pyspark import SparkContext
from pyspark import SparkFiles
import datetime
from confluent_kafka.avro.cached_schema_registry_client import CachedSchemaRegistryClient


In [10]:
def getJsonSchema(schemaId, schemasById, registrySchemaUrl):
        jsonSchema = schemasById.get(schemaId)
        if jsonSchema is None:
            src = SchemaRegistryClient(registrySchemaUrl)
            schema = src.get_by_id(schemaId)
            jsonSchema = json.dumps(schema.flat_schema)
            schemasById[schemaId] = jsonSchema
        return jsonSchema

In [50]:
def getJsonSchemaLatestCached(registrySchemaUrl, topicName):
        src = CachedSchemaRegistryClient(registrySchemaUrl)
        schema = src.get_latest_schema(topicName)[1]
        
#         jsonSchema = json.dumps(schema.schema)
        return str(schema)

In [29]:
def install(package):
    subprocess.check_call([sys.executable, "-m", "pip", "install", package])
    
def avroJsonSchema(schemaId, payload):
#         import subprocess
#         install("schema_registry")
#         install("python-schema-registry-client")
#         install("pandas")
        from schema_registry.client import SchemaRegistryClient, schema
        import json
        
        src = SchemaRegistryClient("http://52.35.125.165:8081")
        schema = src.get_by_id(schemaId)
        jsonSchema = json.dumps(schema.flat_schema)
        return json.dumps(from_avro(payload, jsonSchema))
#         return jsonSchema

getJsonSchema_udf = udf(avroJsonSchema, StringType())

def getJsonSchemaLatest(registrySchemaUrl, topicName):
        src = SchemaRegistryClient(registrySchemaUrl)
        schema = src.get_schema(subject=topicName).schema
        jsonSchema = json.dumps(schema.flat_schema)
        return jsonSchema

In [14]:
kafka = kafka.filter("value is NOT NULL")

sd = datetime.datetime.strptime("03-03-2022", "%d-%m-%Y")

kafka = kafka.filter(kafka['timestamp'] >= sd)
    
byteArrayToLong = fn.udf(lambda x: int.from_bytes(x, byteorder='big', signed=False), LongType())

In [15]:
df = kafka.withColumn('valuenew', kafka['value'])
df = df.withColumn("schemaId", byteArrayToLong(fn.substring("valuenew", 2, 4))) \
                    .withColumn("payload", fn.expr("substring(valuenew, 6, length(valuenew)-5)"))



In [16]:
df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)
 |-- valuenew: binary (nullable = true)
 |-- schemaId: long (nullable = true)
 |-- payload: binary (nullable = true)



In [49]:
src = CachedSchemaRegistryClient("http://52.35.125.165:8081")
schema = src.get_latest_schema("qatest12nonfes.public.field_data_3_2022-value")

print(str(schema[1]))
dfSchema = getJsonSchemaLatestCached("http://52.35.125.165:8081", "qatest12nonfes.public.field_data_3_2022-value")
df = df.withColumn("payload", fn.expr("substring(valuenew, 6, length(valuenew)-5)"))
df = df.withColumn('value', from_avro("payload", dfSchema))
df = df.drop("payload")

/tmp/ipykernel_4077/2689471534.py:1: DeprecationWarning: CachedSchemaRegistry constructor is being deprecated. Use CachedSchemaRegistryClient(dict: config) instead. Existing params ca_location, cert_location and key_location will be replaced with their librdkafka equivalents as keys in the conf dict: `ssl.ca.location`, `ssl.certificate.location` and `ssl.key.location` respectively
  src = CachedSchemaRegistryClient("http://52.35.125.165:8081")


{"type": "record", "connect.name": "qatest12nonfes.public.field_data_3_2022.Envelope", "name": "Envelope", "namespace": "qatest12nonfes.public.field_data_3_2022", "fields": [{"type": ["null", {"type": "record", "connect.name": "qatest12nonfes.public.field_data_3_2022.Value", "name": "Value", "namespace": "qatest12nonfes.public.field_data_3_2022", "fields": [{"type": "long", "name": "id"}, {"type": ["null", "string"], "name": "value", "default": null}, {"type": "int", "name": "field_attribute_master_id"}, {"type": ["null", "int"], "name": "position_id", "default": null}, {"type": ["null", "int"], "name": "parent_id", "default": null}, {"type": "long", "name": "job_transaction_id"}, {"type": ["null", "int"], "name": "company_id", "default": null}, {"type": ["null", {"type": "long", "connect.version": 1, "connect.name": "org.apache.kafka.connect.data.Timestamp", "logicalType": "timestamp-millis"}], "name": "created_at", "default": null}]}], "name": "before", "default": null}, {"type": ["n

In [51]:
dfSchema = getJsonSchemaLatest("http://52.35.125.165:8081", "qatest12nonfes.public.field_data_3_2022-value")
df = df.withColumn("payload", fn.expr("substring(valuenew, 6, length(valuenew)-5)"))
df = df.withColumn('value', from_avro("payload", dfSchema))
df = df.drop("payload")

/tmp/ipykernel_4077/2156750714.py:2: DeprecationWarning: CachedSchemaRegistry constructor is being deprecated. Use CachedSchemaRegistryClient(dict: config) instead. Existing params ca_location, cert_location and key_location will be replaced with their librdkafka equivalents as keys in the conf dict: `ssl.ca.location`, `ssl.certificate.location` and `ssl.key.location` respectively
  src = CachedSchemaRegistryClient(registrySchemaUrl)


In [52]:
dfSchema

'{"type": "record", "connect.name": "qatest12nonfes.public.field_data_3_2022.Envelope", "name": "Envelope", "namespace": "qatest12nonfes.public.field_data_3_2022", "fields": [{"type": ["null", {"type": "record", "connect.name": "qatest12nonfes.public.field_data_3_2022.Value", "name": "Value", "namespace": "qatest12nonfes.public.field_data_3_2022", "fields": [{"type": "long", "name": "id"}, {"type": ["null", "string"], "name": "value", "default": null}, {"type": "int", "name": "field_attribute_master_id"}, {"type": ["null", "int"], "name": "position_id", "default": null}, {"type": ["null", "int"], "name": "parent_id", "default": null}, {"type": "long", "name": "job_transaction_id"}, {"type": ["null", "int"], "name": "company_id", "default": null}, {"type": ["null", {"type": "long", "connect.version": 1, "connect.name": "org.apache.kafka.connect.data.Timestamp", "logicalType": "timestamp-millis"}], "name": "created_at", "default": null}]}], "name": "before", "default": null}, {"type": ["

In [53]:
df.show(1)

+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+
|                 key|               value|               topic|partition|offset|           timestamp|timestampType|            valuenew|schemaId|
+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+
|[00 00 00 0E 6C 9...|[, [1219656,, 21,...|qatest12nonfes.pu...|        2|   100|2022-03-03 10:25:...|            0|[00 00 00 0E 6D 0...|    3693|
+--------------------+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+
only showing top 1 row



22/03/09 04:08:12 ERROR TaskSchedulerImpl: Lost executor 1 on 172.22.0.4: worker lost
22/03/09 04:08:12 ERROR TaskSchedulerImpl: Lost executor 0 on 172.22.0.5: worker lost
22/03/09 04:30:42 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/03/09 04:30:42 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:716)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:152)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:258)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:168)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.nett

In [18]:
schemasByIdDict = {}

dfAllSchemas = df.select("schemaId").distinct()
schemaRowList = dfAllSchemas.collect()
print(schemaRowList)

In [22]:
dfBySchemaId.show(1)

+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+--------------------+--------------------+
|                 key|               topic|partition|offset|           timestamp|timestampType|            valuenew|schemaId|             payload|               value|
+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+--------------------+--------------------+
|[00 00 00 07 32 B...|qatest12nonfes.pu...|        1|195410|2022-03-03 10:26:...|            0|[00 00 00 0E 99 0...|    3737|[00 02 BC A3 2C B...|[, [362718, 53657...|
+--------------------+--------------------+---------+------+--------------------+-------------+--------------------+--------+--------------------+--------------------+
only showing top 1 row



In [ ]:
# jsonSchema = getJsonSchema(3737, schemasByIdDict,"http://52.35.125.165:8081")
# currentValueSchemaId = 3737
# currentValueSchema = jsonSchema
# dfBySchemaId = df.where(df.schemaId == currentValueSchemaId)
# dfBySchemaId = dfBySchemaId.drop("value")


# dfBySchemaId = dfBySchemaId.withColumn("value", from_avro("payload", currentValueSchema))
# # dfBySchemaId = dfBySchemaId.drop("payload")

In [ ]:
for schemaRow in schemaRowList:
    jsonSchema = getJsonSchema(schemaRow.schemaId, schemasByIdDict,"http://52.35.125.165:8081")
    currentValueSchemaId = schemaRow.schemaId
    currentValueSchema = jsonSchema
    dfBySchemaId = df.where(df.schemaId == currentValueSchemaId)
    dfBySchemaId = dfBySchemaId.drop("value")
    dfBySchemaId = dfBySchemaId.withColumn("value", from_avro("payload", currentValueSchema))
    dfBySchemaId = dfBySchemaId.drop("payload")

In [79]:
display(dfBySchemaId)

DataFrame[key: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int, valuenew: binary, schemaId: bigint, value: struct<before:struct<id:bigint,user_id:int,hub_id:int,city_id:int,company_id:int,date:timestamp,app_version:string,gps_kms:double,odometer_kms:double,halt_duration:double,travel_duration:double,avg_speed:double,max_speed:double,official_sms_sent_count:int,personal_sms_sent_count:int,official_call_incoming_count:int,official_call_outgoing_count:int,official_call_incoming_duration:int,official_call_outgoing_duration:int,personal_call_incoming_count:int,personal_call_outgoing_count:int,personal_call_incoming_duration:int,personal_call_outgoing_duration:int,cug_call_incoming_count:int,cug_call_outgoing_count:int,cug_call_incoming_duration:int,cug_call_outgoing_duration:int,last_location_datetime:timestamp,last_lat:double,last_lng:double,last_speed:double,last_battery:int,pending_count:int,deliver_count:int,fail_count:int,cash_collected:d

In [19]:
!pip install python-schema-registry-client

from pyspark.sql.avro.functions import from_avro
from pyspark.sql.functions import col
from pyspark.sql import functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
import base64
import gzip
import _datetime
from schema_registry.client import SchemaRegistryClient
import json
import sys
import datetime
from pyspark.sql.functions import max as sparkMax
from pyspark.sql.functions import broadcast
from delta.tables import *

# import org.apache.spark.sql.streaming.ProcessingTime

In [20]:
def get_partition_query(partition_column: str, primary_key: str):

        if not primary_key or not partition_column:
            return "updatesTable.id = targetTable.id"
        
#         query = "updatesTable.{0} = targetTable.{0}".format(primary_key)
        
        query = ""
        
        primary_keyA = []
    
        pkey_array = primary_key.split(",")
        for partitions in pkey_array:
            partitions = partitions.strip()
            if query == "":
              query += "updatesTable.{0} = targetTable.{0}".format(partitions)
            else:
              query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
              
            primary_keyA.append(partitions)
        
#         primary_key = primary_key.strip()
        partitions_array = partition_column.split(",")
        for partitions in partitions_array:
            partitions = partitions.strip()
            if partitions not in primary_keyA:
                query += " and updatesTable.{0} = targetTable.{0}".format(partitions)
        return query

    
get_partition_query("", "company_id")

'updatesTable.id = targetTable.id'

In [22]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType
from delta.tables import *
from pyspark.sql import DataFrame
  
def getTableColumn(my_schema):
    tableList = []
    col = []
    inputTable = "updatesTable."
    targetTable = "targetTable."

    ignoreColumns = ["table_key_ts", "table_delete", "table_key"]

    for field in my_schema.fields:
        if field.name == "after":
            for f3 in field.dataType:
                te = f3.name + " " + f3.dataType.typeName()
                if f3.name not in ignoreColumns:
                    tableList.append(te)
                    col.append(f3.name)
        else:
            te = field.name + " " + field.dataType.typeName()
            if field.name not in ignoreColumns:
                tableList.append(te)
                col.append(field.name)
    res = ""
    insertResInp = ""
    insertResOut = ""
    merge = ""
    for i in range(len(tableList)):
        if i == (len(tableList) - 1):
            res += tableList[i]
            insertResInp += col[i]
            insertResOut += inputTable + col[i]
            merge += targetTable + col[i] + " = " + inputTable + col[i]
        else:
            res += tableList[i] + ","
            insertResInp += col[i] + ","
            insertResOut += inputTable + col[i] + ","
            merge += targetTable + col[i] + " = " + inputTable + col[i] + ","
    ans = [res, insertResInp, insertResOut, merge]
    
    
    print(ans)
    return ans

def groupk(partition):
      from itertools import groupby
      
      partCount = 1
      l = []
      for key, group in groupby(sorted(partition, key = lambda x: x["table_key"]), lambda x: x["table_key"]):
        a = []
        k = None
        temp = -1

        for thing in group:
          if temp < int(thing["offset"]):
            temp = thing["offset"]
            k = thing

#         yield k
        l.append(k)
        partCount += 1
    
      return l
    
def upsertToDelta(microBatchOutputDF):
        df5 = spark.sparkContext._jvm.com.example.Hello.add(spark._jsparkSession, "34.220.52.64:9092,52.35.125.165:9092", "datalake-qa-91")
        microBatchOutputDF = microBatchOutputDF.drop("valuenew")
        
        microBatchOutputDF = microBatchOutputDF.withColumn('valueConsumerKafka', microBatchOutputDF['value'])
        microBatchOutputDF = microBatchOutputDF.drop("value")
        
        microBatchOutputDF.show(1)
        
        
        microBatchOutputDF = microBatchOutputDF.withColumn("table_delete", F.when(
            col("valueConsumerKafka.before").isNotNull and col("valueConsumerKafka.op").isNotNull and col("valueConsumerKafka.op").contains("d"),
            True).otherwise(False))
        
        
        if True:
          microBatchOutputDF = microBatchOutputDF.withColumn("table_key", F.when(col("table_delete") == True,
                                                                                 microBatchOutputDF[
                                                                                     "valueConsumerKafka.before.{}".format(
                                                                                         "id")]).otherwise(
              microBatchOutputDF["valueConsumerKafka.after.{}".format("id")]))

        
          
        
        print("starting point")

        print(datetime.datetime.now())
        
        print("total partitions are")
        print(microBatchOutputDF.rdd.getNumPartitions())
        

        dfrdd = microBatchOutputDF.rdd.mapPartitions(groupk)
        dfByTopic = spark.createDataFrame(dfrdd, schema = microBatchOutputDF.schema)
        
        print(dfByTopic.rdd.getNumPartitions())
        
        print(datetime.datetime.now())
        
#         display(dfByTopic)

        dfByTopicDeleted = dfByTopic.filter(F.col("table_delete") == True)
        dfByTopicDeleted = dfByTopicDeleted.select("valueConsumerKafka.before.*", "table_delete", "table_key", "offset")

        dfByTopicDeleted = dfByTopicDeleted.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicDeleted = dfByTopicDeleted.drop("offset")

        dfByTopicInsert = dfByTopic.filter(F.col("table_delete") == False)
        dfByTopicInsert = dfByTopicInsert.select("valueConsumerKafka.after.*", "table_delete", "table_key", "offset")
        # display(df)
        dfByTopicInsert = dfByTopicInsert.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicInsert = dfByTopicInsert.drop("offset")
        
        if True:
            partition_column_derive_from_column_split = "created_at_part,created_at".split(",")
            new_partition_key = partition_column_derive_from_column_split[0]
            new_partition_value = partition_column_derive_from_column_split[1]
            
        print(new_partition_value)
        
        if True:
            dfByTopicDeleted = dfByTopicDeleted.withColumn(new_partition_key,to_date(col(new_partition_value)))
            
            dfByTopicInsert = dfByTopicInsert.withColumn(new_partition_key,to_date(col(new_partition_value)))
            
#         if True:
#             my_schema = dfByTopicInsert.schema
#             res = getTableColumn(my_schema)
# #             dfByTopic._jdf.sparkSession().sql("use {}".format(self.database))
#             if True:
#                 print("table created")
#                 dfByTopic._jdf.sparkSession().sql(
#                     "create table if not exists {} ( {} ) using delta PARTITIONED BY ({}) location '{}' ".format("user_summary_old",
#                                                                                                                  res[0],
#                                                                                                                  "company_id,created_at_part",
#                                                                                                                  "user_summary_old"))

#             table_exists = True

        dfByTopicInsert = dfByTopicInsert.drop("table_delete")
        dfByTopicInsert = dfByTopicInsert.drop("table_key")
        
        FullTableName = "field_data"

        print(FullTableName)
        
        deltaTable=DeltaTable.forName(spark,FullTableName)

        if True:
#             return dfByTopicInsert
#             dfByTopicInsert.write.format("delta").mode("overwrite").save("field_data")
#             query = get_partition_query("company_id,created_at_part", "id")
            
            print("checkpoint 1")
            
            deltaTable.alias("targetTable").merge(
                dfByTopicDeleted.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedDelete() \
              .execute()


            deltaTable.alias("targetTable").merge(
                dfByTopicInsert.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()
        
        print("after merge")

        #logger.debug("after merge")
        
        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))


In [23]:
# dfBySchemaId.printSchema()
dfBySchemaId = df
dfa = upsertToDelta(dfBySchemaId)

datalake-qa-91

+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+
|                 key|               topic|partition|offset|           timestamp|timestampType|schemaId|  valueConsumerKafka|
+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+
|[00 00 00 0E 6C 9...|qatest12nonfes.pu...|        2|   100|2022-03-03 10:25:...|            0|    3693|[, [1219656,, 21,...|
+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+
only showing top 1 row

starting point
2022-03-08 07:31:38.852534
total partitions are


Exception in thread "serve-DataFrame" java.net.SocketTimeoutException: Accept timed out
	at java.net.PlainSocketImpl.socketAccept(Native Method)
	at java.net.AbstractPlainSocketImpl.accept(AbstractPlainSocketImpl.java:409)
	at java.net.ServerSocket.implAccept(ServerSocket.java:560)
	at java.net.ServerSocket.accept(ServerSocket.java:528)
	at org.apache.spark.security.SocketAuthServer$$anon$1.run(SocketAuthServer.scala:58)


140
140
2022-03-08 07:32:14.255257
created_at
field_data


checkpoint 1
after merge
2022-03-08 07:36:28.878966


In [24]:
from pyspark.sql.types import StructType, StructField, IntegerType, LongType, StringType, FloatType
from delta.tables import *
from pyspark.sql import DataFrame
  
def getTableColumn(my_schema):
    tableList = []
    col = []
    inputTable = "updatesTable."
    targetTable = "targetTable."

    ignoreColumns = ["table_key_ts", "table_delete", "table_key"]

    for field in my_schema.fields:
        if field.name == "after":
            for f3 in field.dataType:
                te = f3.name + " " + f3.dataType.typeName()
                if f3.name not in ignoreColumns:
                    tableList.append(te)
                    col.append(f3.name)
        else:
            te = field.name + " " + field.dataType.typeName()
            if field.name not in ignoreColumns:
                tableList.append(te)
                col.append(field.name)
    res = ""
    insertResInp = ""
    insertResOut = ""
    merge = ""
    for i in range(len(tableList)):
        if i == (len(tableList) - 1):
            res += tableList[i]
            insertResInp += col[i]
            insertResOut += inputTable + col[i]
            merge += targetTable + col[i] + " = " + inputTable + col[i]
        else:
            res += tableList[i] + ","
            insertResInp += col[i] + ","
            insertResOut += inputTable + col[i] + ","
            merge += targetTable + col[i] + " = " + inputTable + col[i] + ","
    ans = [res, insertResInp, insertResOut, merge]
    
    
    print(ans)
    return ans

def groupk(partition):
      from itertools import groupby
      
      partCount = 1
      l = []
      for key, group in groupby(sorted(partition, key = lambda x: x["table_key"]), lambda x: x["table_key"]):
        a = []
        k = None
        temp = -1

        for thing in group:
          if temp < int(thing["offset"]):
            temp = thing["offset"]
            k = thing

#         yield k
        l.append(k)
        partCount += 1
    
      return l
    
def upsertToDelta2(kafka, batchId):
        byteArrayToLong = fn.udf(lambda x: int.from_bytes(x, byteorder='big', signed=False), LongType())
        df = kafka.withColumn('valuenew', kafka['value'])
        df = df.withColumn("schemaId", byteArrayToLong(fn.substring("valuenew", 2, 4))) \
                            .withColumn("payload", fn.expr("substring(valuenew, 6, length(valuenew)-5)"))


        jsonSchema = getJsonSchema(3737, schemasByIdDict,"http://52.35.125.165:8081")
        currentValueSchemaId = 3737
        currentValueSchema = jsonSchema
        dfBySchemaId = df.where(df.schemaId == currentValueSchemaId)
        dfBySchemaId = dfBySchemaId.drop("value")


        dfBySchemaId = dfBySchemaId.withColumn("value", from_avro("payload", currentValueSchema))
        
        microBatchOutputDF = dfBySchemaId

        df5 = spark.sparkContext._jvm.com.example.Hello.add(spark._jsparkSession, "34.220.52.64:9092,52.35.125.165:9092", "datalake-qa-91")
        microBatchOutputDF = microBatchOutputDF.drop("valuenew")
        
        microBatchOutputDF = microBatchOutputDF.withColumn('valueConsumerKafka', microBatchOutputDF['value'])
        microBatchOutputDF = microBatchOutputDF.drop("value")
        
        microBatchOutputDF.show(1)
        
        
        microBatchOutputDF = microBatchOutputDF.withColumn("table_delete", F.when(
            col("valueConsumerKafka.before").isNotNull and col("valueConsumerKafka.op").isNotNull and col("valueConsumerKafka.op").contains("d"),
            True).otherwise(False))
        
        
        if True:
          microBatchOutputDF = microBatchOutputDF.withColumn("table_key", F.when(col("table_delete") == True,
                                                                                 microBatchOutputDF[
                                                                                     "valueConsumerKafka.before.{}".format(
                                                                                         "id")]).otherwise(
              microBatchOutputDF["valueConsumerKafka.after.{}".format("id")]))

        
          
        
        print("starting point")

        print(datetime.datetime.now())
        
        print("total partitions are")
        print(microBatchOutputDF.rdd.getNumPartitions())
        

        dfrdd = microBatchOutputDF.rdd.mapPartitions(groupk)
        dfByTopic = spark.createDataFrame(dfrdd, schema = microBatchOutputDF.schema)
        
        print(dfByTopic.rdd.getNumPartitions())
        
        print(datetime.datetime.now())
        
#         display(dfByTopic)

        dfByTopicDeleted = dfByTopic.filter(F.col("table_delete") == True)
        dfByTopicDeleted = dfByTopicDeleted.select("valueConsumerKafka.before.*", "table_delete", "table_key", "offset")

        dfByTopicDeleted = dfByTopicDeleted.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicDeleted = dfByTopicDeleted.drop("offset")

        dfByTopicInsert = dfByTopic.filter(F.col("table_delete") == False)
        dfByTopicInsert = dfByTopicInsert.select("valueConsumerKafka.after.*", "table_delete", "table_key", "offset")
        # display(df)
        dfByTopicInsert = dfByTopicInsert.drop("valueConsumerKafka", "kafka-key", "schemaId")
        dfByTopicInsert = dfByTopicInsert.drop("offset")
        
        if True:
            partition_column_derive_from_column_split = "created_at_part,created_at".split(",")
            new_partition_key = partition_column_derive_from_column_split[0]
            new_partition_value = partition_column_derive_from_column_split[1]
            
        print(new_partition_value)
        
        if True:
            dfByTopicDeleted = dfByTopicDeleted.withColumn(new_partition_key,to_date(col(new_partition_value)))
            
            dfByTopicInsert = dfByTopicInsert.withColumn(new_partition_key,to_date(col(new_partition_value)))
            
#         if True:
#             my_schema = dfByTopicInsert.schema
#             res = getTableColumn(my_schema)
# #             dfByTopic._jdf.sparkSession().sql("use {}".format(self.database))
#             if True:
#                 print("table created")
#                 dfByTopic._jdf.sparkSession().sql(
#                     "create table if not exists {} ( {} ) using delta PARTITIONED BY ({}) location '{}' ".format("user_summary_old",
#                                                                                                                  res[0],
#                                                                                                                  "company_id,created_at_part",
#                                                                                                                  "user_summary_old"))

#             table_exists = True

        dfByTopicInsert = dfByTopicInsert.drop("table_delete")
        dfByTopicInsert = dfByTopicInsert.drop("table_key")
        
        FullTableName = "user_summary_old"

        print(FullTableName)
        
        deltaTable=DeltaTable.forName(spark,FullTableName)

        if True:
#             return dfByTopicInsert
#             dfByTopicInsert.write.format("delta").mode("overwrite").save("user_summary_old")
#             query = get_partition_query("company_id,created_at_part", "id")
            
            print("checkpoint 1")
            
#             deltaTable.alias("targetTable").merge(
#                 datau.alias("updatesTable"),
#                 "{0}".format("updatesTable.id = targetTable.id")) \
#               .whenMatchedUpdateAll() \
#               .whenNotMatchedInsertAll().execute()

            deltaTable.alias("targetTable").merge(
                dfByTopicDeleted.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedDelete() \
              .execute()


            deltaTable.alias("targetTable").merge(
                dfByTopicInsert.alias("updatesTable"),
                "{0}".format(query)) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()
        
        print("after merge")

        #logger.debug("after merge")
        
        print(datetime.datetime.now())
        
        #logger.debug(str(datetime.datetime.now()))

        
kafka = spark.readStream \
            .format("kafka") \
            .option("kafka.bootstrap.servers", "34.220.52.64:9092,52.35.125.165:9092") \
            .option("subscribe", "qatest12nonfes.public.user_summary_old") \
            .option("startingOffsets", "earliest") \
            .option("maxOffsetsPerTrigger", 1000000) \
            .option("failOnDataLoss", False) \
            .load()


kafka = kafka.filter("value is NOT NULL")

ssc = kafka \
            .writeStream \
            .queryName("{0}".format("user_summary_old")) \
            .option("checkpointLocation", "cp/raw/") \
            .trigger(processingTime='10 seconds') \
            .foreachBatch(upsertToDelta2).start()

ssc.awaitTermination()

Query started: 61ee83fb-6c47-4e7d-bd63-0c0c2ac44e7f


22/03/08 07:42:21 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


datalake-qa-91

+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+--------------------+
|                 key|               topic|partition|offset|           timestamp|timestampType|schemaId|             payload|  valueConsumerKafka|
+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+--------------------+
|[00 00 00 07 32 F...|qatest12nonfes.pu...|        9|185099|2022-03-07 14:27:...|            0|    3737|[00 02 FE CB 2C F...|[, [365311, 3834,...|
+--------------------+--------------------+---------+------+--------------------+-------------+--------+--------------------+--------------------+
only showing top 1 row

starting point
2022-03-08 07:43:58.901645
total partitions are
1
1
2022-03-08 07:43:59.119850
created_at
user_summary_old


checkpoint 1
after merge
2022-03-08 07:44:16.648590


...

Query made progress: {qatest12nonfes.public.user_summary_old-9=OffsetAndMetadata{offset=185963, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-7=OffsetAndMetadata{offset=191258, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-8=OffsetAndMetadata{offset=189877, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-5=OffsetAndMetadata{offset=171068, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-6=OffsetAndMetadata{offset=194807, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-3=OffsetAndMetadata{offset=201131, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-4=OffsetAndMetadata{offset=169495, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-1=OffsetAndMetadata{offset=199521, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-2=OffsetAndMetadata{offset=174686, leaderEpoch=null, metadata=''}, qatest12nonfes.public.us

22/03/08 07:49:19 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
22/03/08 07:49:19 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:716)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:152)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:258)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:168)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:203)
	at org.apache.spark.rpc.netty.Inbox.process(Inbox.scala:100)
	at org.apache.spark.rpc.netty.MessageLoop.org$apache$spark$rpc$netty$Mess

Query made progress: {qatest12nonfes.public.user_summary_old-9=OffsetAndMetadata{offset=185963, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-7=OffsetAndMetadata{offset=191258, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-8=OffsetAndMetadata{offset=189877, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-5=OffsetAndMetadata{offset=171068, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-6=OffsetAndMetadata{offset=194807, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-3=OffsetAndMetadata{offset=201131, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-4=OffsetAndMetadata{offset=169495, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-1=OffsetAndMetadata{offset=199521, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-2=OffsetAndMetadata{offset=174686, leaderEpoch=null, metadata=''}, qatest12nonfes.public.us

22/03/08 07:49:20 ERROR MicroBatchExecution: Query user_summary_old [id = 61ee83fb-6c47-4e7d-bd63-0c0c2ac44e7f, runId = 7ff08652-fc4e-4e0b-86df-98a8f7ff8258] terminated with error
py4j.Py4JException: An exception was raised by the Python Proxy. Return Message: Traceback (most recent call last):
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gateway.py", line 2442, in _call_proxy
    return_value = getattr(self.pool[obj_id], method)(*params)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/sql/utils.py", line 210, in call
    raise e
  File "/usr/local/lib/python3.9/dist-packages/pyspark/sql/utils.py", line 207, in call
    self.func(DataFrame(jdf, self.sql_ctx), batch_id)
  File "/tmp/ipykernel_1978/3990817060.py", line 91, in upsertToDelta2
    microBatchOutputDF.show(1)
  File "/usr/local/lib/python3.9/dist-packages/pyspark/sql/dataframe.py", line 440, in show
    print(self._jdf.showString(n, 20, vertical))
  File "/usr/local/lib/python3.9/dist-packages/py4j/java_gat

Query made progress: {qatest12nonfes.public.user_summary_old-9=OffsetAndMetadata{offset=185963, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-7=OffsetAndMetadata{offset=191258, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-8=OffsetAndMetadata{offset=189877, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-5=OffsetAndMetadata{offset=171068, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-6=OffsetAndMetadata{offset=194807, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-3=OffsetAndMetadata{offset=201131, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-4=OffsetAndMetadata{offset=169495, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-1=OffsetAndMetadata{offset=199521, leaderEpoch=null, metadata=''}, qatest12nonfes.public.user_summary_old-2=OffsetAndMetadata{offset=174686, leaderEpoch=null, metadata=''}, qatest12nonfes.public.us

In [33]:
import pyspark.sql.functions as fn
data = spark.range(1,5)
data.write.format("delta").mode("overwrite").save("delta_sample1")

In [32]:
dfa.show(1)
# dfa.write.format("delta").mode("overwrite").save("user_summary_old")

In [2]:
spark.sql("CREATE TABLE " + "field_data" + " USING DELTA LOCATION '" + "field_data" + "'")

DataFrame[]

In [36]:
deltaTable=DeltaTable.forName(spark,"delta_sample1")

In [9]:
# print(deltaTable)
df1 = spark.sql("select count(*) from field_data limit 5")

In [38]:
print(data)

DataFrame[id: bigint]


In [10]:
df1.show(1)

+--------+
|count(1)|
+--------+
|   10678|
+--------+



In [40]:
datau = spark.range(3,8)

deltaTable.alias("targetTable").merge(
                datau.alias("updatesTable"),
                "{0}".format("updatesTable.id = targetTable.id")) \
              .whenMatchedUpdateAll() \
              .whenNotMatchedInsertAll().execute()

In [42]:
df1 = spark.sql("select * from delta_sample1")
df1.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  6|
|  5|
|  7|
+---+

